In [12]:
# importing packages

import pandas as pd
import numpy as np

from hashlib import sha256
import json

import matplotlib.pyplot as plt

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Background

Depois de meses a fazer, candidatures finalmente consegues um estágio de Verão no laboratório **LabOfLife (LOL)**

Começas o estágio. És inserido numa equipa de laboratório que analisa proteínas e as suas propriedades. Nos primeiros dias és introduzido aos diversos projetos que o laboratório tem e também à equipa onde te vais inserir, e mal podes esperar por começar a aprender e a aplicar os teus conhecimentos.

Certo dia chegas ao laboratório bem cedo, e mal entras na sala ficas surpreso por ver toda a tua equipa já lá, e ainda por cima o ambiente parece pesado... Preocupado, chegas-te a uma colega e perguntas o que se está a passar. Ela diz-te: 


> "Estamos a analisar as fosforilações presentes num proteoma de alfa-casína. Para isso fizémos uma experiência e recolhemos os dados todos para o ficheiro `data/dados.csv`. No entanto passámos a noite toda a tentar terminar a nossa análise das fosforilações, mas a única coisa que conseguimos fazer foi uma função que está no ficheiro `utils.py` que extrai só os eventos de fosforilações da coluna que contém todas as modificações que aconteceram às proteínas"

Aprontamente, tu ofereces a tua ajuda:

> "Não te preocupes. Eu fiz um workshop de Pandas e Python organizado pelo ANEEB e consigo-vos ajudar, explica-me melhor a situação e eu resolvo isto em menos de 1h."

Os teus colegas agradecem-te, explicam a situação de forma detalhada e vão descansar para casa.


**background**:
> Estamos a testar umas nanopartículas (IMAC – immobilized ion metal afinity chromatography) que servem para pré-concentrar os fosfopeptidos. Queremos testar se a nossa concentração foi bem feita olhando para as fosforilações das sequencias de aminoácidos.

**Objetivo**:

> Analisar o número total de fosforilações por sequência de aminoácidos, bem como o número de fosforilações que ocorrem nos aminoácidos S, T e Y. Queremos verificar se rondamos a distribuição esperada: 80% de S, 18% de T e 2% de Y. Caso os resultados apontem para números muito diferentes estamos perante selectividades diferentes.


**Dados**:

> Os dados com que vais trabalhar resultam de análise por espectrometria de massa da pré-concentração usando IMACs de titânio, európio e samário por um programa chamado MASCOT. O dataset contém muita informação, mas para esta análise só nos interessa olhar para a **sequência** de aminoácidos e as fosforilações que estão presentes na coluna de **modificações**.

# Exercise 1 - Prepare de data

## Exercise 1.1 - Read the data

In [6]:
# df = ... # uncomment to provide an answer

# df.head() # uncomment to inspect the data

In [7]:
df=pd.read_csv('data/dados_fosforilacoes_aminoacidos.csv')

Evaluation cell below

In [14]:
# don't forget to hide contents by default
assert df.shape==(2541,20), "Oops!"

## Exercise 1.2 - Select relevant columns

There's a lot of information on this dataset, but you remember that the problem consists of analyzing only the phosphorylation events out of all **Modifications** that occurred on the **Sequence** of aminoacids. Look for the 2 columns that contain this information and create a new dataframe that only contains these columns.

In [2]:
#relevant_columns = [... , ...] # uncomment to provide an answer. hint: this should be a list of two strings

#phospho_df = df[relevant_columns] # uncomment to provide an answer
#phospho_df.head() 

In [50]:
relevant_columns = ["Sequence", 'Modifications'] # uncomment to provide an answer. hint: this should be a list of two strings

phospho_df = df[relevant_columns] # uncomment to provide an answer
phospho_df.head() 

,Sequence,Modifications
0,K.TVDMESTEVFTK.K,Acetyl: 1; Oxidation: 4; Phospho: 6
1,K.VPQLEIVPNSAEER.L,Phospho: 10
2,K.VPQLEIVPNSAEER.L,Phospho: 10
3,K.VPQLEIVPNSAEER.L,Phospho: 10
4,K.VPQLEIVPNSAEER.L,Phospho: 10


In [52]:
hash_df = b'\xe9\x9c\xc8\x8d9\xc9\x9e\x1d\xf2\xe4B\xc6\xc1S+\x9aPt\xa8\x89\xc8\x00[!\xd8:-\xfa~\xb0\xae\xcd'

assert phospho_df.shape==(2541,2), "Oops!"
assert sha256(phospho_df.reindex(sorted(phospho_df.columns), axis=1).to_dict().__str__().encode()).digest() == hash_df

# Exercise 2

## Exercise 2.1 - Clean the Sequence column

We have the data we need. The `Modifications` column contains the position of each reaction in the sequence. However, the `Sequence` column needs some cleaning because each aminoacid sequence is in the middle of two full stops: "\<Letter\>.**\<sequence\>**.\<Letter\>"

**Objective**: create a function that receives a sequence in its raw form and extracts only the actual sequence.

In [54]:
def clean_sequence(sequence):
    """
    Extract the actual aminoacid sequence from a MASCOT sequence.
    
    Parameters
    ----------
    
    sequence : str
        An sequence produced by the program MASCOT.
        
    Returns
    -------
    
    amino_sequence : str
        The aminoacid sequence
    
    Examples
    --------
    
    sequence = 'K.VPQLEIVPNSAEER.L'
    clean_sequence(sequence)
    >>> 'VPQLEIVPNSAEER'
    
    """
    
    # amino_sequence = ...
    return amino_sequence

# try it yourself!
# example = 'K.VPQLEIVPNSAEER.L' # uncomment to test the function
# clean_sequence(example) # uncomment to test the function